## 환경 설정 & 라이브러리

In [15]:
# 필수 라이브러리 불러오기
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
import os
from dotenv import load_dotenv

# .env 파일 로드 (OpenAI API Key 불러오기)
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

if api_key:
    print("OpenAI API Key 로드 성공!")
else:
    print("OpenAI API Key가 없습니다. .env 파일을 확인하세요.")


OpenAI API Key 로드 성공!


## 벡터DB 불러오기

In [33]:
# 기존 벡터 DB 로드
vectorstore = FAISS.load_local('../data/db/faiss', OpenAIEmbeddings(), allow_dangerous_deserialization=True)

# 대화 기반 검색을 위한 Retriever 설정
retriever = vectorstore.as_retriever()

# Memory 설정
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True, output_key="answer")

# Conversational RAG 설정
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.7)

conversation_rag = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    return_source_documents=False,  
    #output_key="answer"  # 어떤 출력을 기억할지 명확히 지정
)

print("Conversational RAG 설정 완료!")

Conversational RAG 설정 완료!


## 상담 챗봇 프롬프트 최적화

In [34]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """당신은 고민을 들어주고 공감하며 상담을 도와주는 AI 챗봇입니다.  
사용자가 고민을 이야기하면 **친절하고 따뜻한 말투**로 공감해주세요.  
사용자가 편안하게 속마음을 이야기할 수 있도록 친근한 표현을 사용하세요.  

💡 **상담 원칙**  
- 사용자의 감정을 **먼저 인정하고** 따뜻한 말로 반응합니다.  
- 사용자의 고민을 듣고 **같이 공감하며 질문을 던져** 대화를 이어갑니다.  
- 단순한 정보 전달이 아닌, **친근하고 자연스럽게 대화하는 방식**을 사용합니다.  
- RAG 검색 결과를 참고하되, **무조건 정보를 나열하지 말고** 사용자의 감정에 초점을 맞춰 답변합니다.  

🚫 **이렇게 답변하지 마세요!**  
- "많은 사람들이 그렇습니다. 상담을 받아보세요." ❌ (너무 일반적)  
- "기분이 안 좋은 이유는 여러 가지가 있습니다." ❌ (공감 부족)  
- "해결 방법은 이러이러합니다." ❌ (강요하는 느낌)  

✅ **좋은 상담 예시**  
👤 사용자: "나 너무 힘들어."  
🤖 챗봇: "그렇군요.. 요즘 많이 지치셨겠어요. 무슨 일이 있으셨나요? 제가 들어드릴게요. 😊"  

👤 사용자: "학교에서 친구랑 싸웠어."  
🤖 챗봇: "저런.. 속상하셨겠어요. 친구와 다투면 마음이 많이 복잡해지죠. 어떤 일 때문에 다투게 되었나요?"  

---

💬 **사용자의 고민:**  
{question}  

📂 **참고할 상담 사례:**  
{context}  

🤖 **챗봇 응답:**  
"""
)


## 상담 챗봇 테스트

In [20]:
# 대화 테스트 
query1 = "나 오늘 너무 우울해."
response1 = conversation_rag.invoke({"question": query1})
print("🤖 챗봇:", response1["answer"])

query2 = "학교에서 친구랑 싸웠어."
response2 = conversation_rag.invoke({"question": query2})
print("🤖 챗봇:", response2["answer"])

query3 = "근데 내 잘못 같기도 해.."
response3 = conversation_rag.invoke({"question": query3})
print("🤖 챗봇:", response3["answer"])


🤖 챗봇: 당신이 우울한 기분을 느끼고 있다니 안타깝습니다. 힘든 날들이 계속되고 있는 것 같네요. 누군가와 이야기하거나 감정을 나누는 것이 도움이 될 수 있습니다. 필요하다면 전문가의 도움도 고려해보세요. 당신의 기분이 나아지기를 바랍니다.
🤖 챗봇: 제가 알기로는 학교에서 친구와 싸운 이유는 여러 가지가 있을 수 있습니다. 예를 들어, 친구가 다른 사람들과 함께 지내는 것을 보고 질투를 느끼거나, 서로의 행동에 대한 오해가 생겨 싸우게 되는 경우가 많습니다. 상황에 따라 다르기 때문에 구체적인 이유는 그 친구들 간의 관계와 상황에 따라 달라질 수 있습니다.
🤖 챗봇: 잘못한 것 같다는 생각은 여러 가지 이유로 드는 경우가 많습니다. 일반적으로는 자신에 대한 높은 기대나 기준이 있을 때, 또는 주변 사람들과의 비교에서 오는 불안감이 원인이 될 수 있습니다. 또한, 과거의 경험이나 타인의 피드백이 영향을 미칠 수도 있습니다. 이런 감정은 자연스러운 것이지만, 이를 계속해서 반복적으로 느끼게 되면 자존감에 부정적인 영향을 미칠 수 있습니다. 전문가와의 상담이나 긍정적인 자기 대화가 도움이 될 수 있습니다.


### 사용자 입력 기반 대화 테스트트

In [31]:
print("💬 고민 상담 챗봇 시작! (종료하려면 'exit' 입력)")

chat_history = []  # 대화 히스토리 저장

while True:
    user_input = input("👤 사용자: ")  # 사용자 입력 받기
    print("👤 사용자:", user_input)  # 사용자 입력 즉시 출력

    if user_input.lower() == "exit":  # 'exit' 입력 시 종료
        print("🔚 상담 챗봇 종료!")
        break

    # RAG 기반 응답 (대화 기록 포함)
    response = conversation_rag.invoke({"question": user_input, "chat_history": chat_history})

    # 챗봇 응답 출력
    print("🤖 챗봇:", response["answer"])

    # 대화 기록 저장
    chat_history.append({"question": user_input, "answer": response["answer"]})

💬 고민 상담 챗봇 시작! (종료하려면 'exit' 입력)
👤 사용자: 나 너무 힘들ㅇ
